In [1]:
from selenium import webdriver 
from selenium.webdriver import Chrome
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
import json
import re, time
import pandas as pd
from collections import defaultdict

In [ ]:
def pull_indeed_job_links(driver, jobs,search):
    '''
    Inputs: Google Chrome browser instance and list for tracking the postings
    Outputs: List of defaultdict dictionaries that are tracking Indeed job postings
    Purpose: Pull in indeed job data scientist links as well as basic information on posting
    '''
    
    job_pane = driver.find_element(By.ID,'jobsearch-Main')#//div[@id=
    job_listings = job_pane.find_elements(By.XPATH,'.//div[@class="job_seen_beacon"]')
    for job_listing in job_listings:
        
        job_link = job_listing.find_element(By.XPATH,'.//a')
        #if re.search('.*Data Scientist.*',job_link.find_element(By.XPATH,'.//span').text):
        job_details = defaultdict()
        job_details['job_description'] = job_link.find_element(By.XPATH,'.//span').text
        job_details['link'] = job_link.get_attribute('href')
        try:
            job_details['company'] = job_listing.find_element(By.XPATH,'.//span[@class="companyName"]').text
        except:
            job_details['company'] = 'Not listed'
        #job_details['company'] = job_listing.find_element(By.XPATH,'.//span[@class="companyName"]').text
        try:
            salary_detail = job_listing.find_element(By.XPATH,'.//div[@class="heading6 tapItem-gutter metadataContainer noJEMChips salaryOnly"]')
            job_details['salary'] = job_listing.find_element(By.XPATH,'.//div[@class="metadata salary-snippet-container"]').text
            job_details['Employment type'] = job_listing.find_element(By.XPATH,'.//div[@class="metadata"]').text

        except:
            job_details['salary'] = None
        job_details['search_term'] = search
        job_details['location'] = job_listing.find_element(By.XPATH,'.//div[@class="companyLocation"]').text.splitlines()[0]
        jobs.append(j

In [ ]:
def open_indeed_link(search_term):
    #print(search_term)
    new_link = f'https://www.indeed.com/q-{search_term}-jobs.html'
    #print(new_link)
    opts = Options()

    driver = webdriver.Chrome(options=opts)#executable_path ='/Users/JoshForster/anaconda3/chromedriver'
    driver.get(new_link)
    jobs=[]
    jobs=pull_indeed_job_links(driver=driver,jobs=jobs,search=search_term)
    return (driver,jobs)

In [ ]:
def paginate(driver,page_num):
    pagination = driver.find_element(By.XPATH,f'//a[@data-testid="pagination-page-{page_num}"]')
    pagination.click()

In [ ]:
search_terms = ['Data-Scientist', 'Data-Engineer', 'Data-Analyst', 'Business-Analyst', 'Data-Architect']
jobs_full = []

In [ ]:
for idx,job_search in enumerate(search_terms):
    print(job_search)
    driver,jobs = open_indeed_link(search_term=job_search)
    jobs_full.extend(jobs)
    for x in range(2,9):
        paginate(driver,x)
        jobs_temp = pull_indeed_job_links(driver=driver,jobs=jobs,search=job_search)
        jobs_full.extend(jobs_temp)
    driver.close()

In [ ]:
df_new = pd.DataFrame(jobs_full)
df_new.head(100)

In [ ]:
df_uniq = df_new.drop_duplicates('link')[~pd.isna(df_new['salary'])]

In [ ]:
df_uniq.to_csv('indeed_scraped_data_2023-10-19.csv')